In [12]:
import numpy as np
from numpy import typing as nptyping

# Numerical Differentiation

### Coefficients of central finite difference approximations<br> 

$$f^{(n)}(x_j)=\frac{1}{h^n}\sum_{q}{c_q}f(x_{j+q})+\mathcal{O}(h^{\nu})$$

|<center>$$n$$|<center>$$\nu$$ |<center>$$c_{-4}$$|<center>$$c_{-3}$$|<center>$$c_{-2}$$|<center>$$c_{-1}$$|<center>$$c_{0}$$|<center>$$c_{+1}$$|<center>$$c_{+2}$$|<center>$$c_{+3}$$|<center>$$c_{+4}$$|
|-|-|-|-|-|-|-|-|-|-|-|
|<center>1|<center>2|<center>0|<center>0|<center>0|<center>-1/2|<center>0|<center>1/2|<center>0|<center>0|<center>0|
|<center>1|<center>4|<center>0|<center>0|<center>1/12|<center>-2/3|<center>0|<center>2/3|<center>-1/12|<center>0|<center>0|
|<center>1|<center>6|<center>0|<center>-1/60|<center>3/20|<center>-3/4|<center>0|<center>3/4|<center>-3/20|<center>1/60|<center>0|
|<center>2|<center>2|<center>0|<center>0|<center>0|<center>1|<center>-2|<center>1|<center>0|<center>0|<center>0|
|<center>2|<center>4|<center>0|<center>0|<center>-1/12|<center>4/3|<center>-5/2|<center>4/3|<center>-1/12|<center>0|<center>0|
|<center>2|<center>6|<center>0|<center>1/90|<center>-3/20|<center>3/2|<center>-49/18|<center>3/2|<center>-3/20|<center>1/90|<center>0|
|<center>3|<center>2|<center>0|<center>0|<center>-1/2|<center>1|<center>0|<center>-1|<center>1/2|<center>0|<center>0|
|<center>3|<center>4|<center>0|<center>1/8|<center>-1|<center>13/8|<center>0|<center>-13/8|<center>1|<center>-1/8|<center>0|
|<center>3|<center>6|<center>-7/240|<center>3/10|<center>-169/120|<center>61/30|<center>0|<center>-61/30|<center>169/120|<center>-3/10|<center>7/240|
|<center>4|<center>2|<center>0|<center>0|<center>1|<center>-4|<center>6|<center>-4|<center>1|<center>0|<center>0|
|<center>4|<center>4|<center>0|<center>-1/6|<center>2|<center>-13/2|<center>28/3|<center>-13/2|<center>2|<center>-1/6|<center>0|
|<center>4|<center>6|<center>7/240|<center>-2/5|<center>169/60|<center>-122/15|<center>91/8|<center>-122/15|<center>169/60|<center>-2/5|<center>7/240|
    
* e.g. 3rd order ($n=3$) derivative with $\mathcal{O}(h^4)$ ($\nu =4$)
    $$f^{(3)}(x_j)=f(x_j)^{\prime\prime\prime}=\frac{f(x_{j-3})-8f(x_{j-2})+13f(x_{j-1})-13f(x_{j+1})+8f(x_{j+2})-f(x_{j+3})}{8h^3}$$

In [13]:
def finite_difference(f, x, space=1e-2, order=1):
    """
    Returns derivative of function f at point x, based on central finite difference.
    Order of remaining terms are above 2.
    Maximum order of 4.
    """
    coeff = np.array(
        [
            [0, 0, 0, -1 / 2, 0, 1 / 2, 0, 0, 0],
            [0, 0, 0, 1, -2, 1, 0, 0, 0],
            [0, 0, -1 / 2, 1, 0, -1, 1 / 2, 0, 0],
            [0, 0, 1, -4, 6, -4, 1, 0, 0],
        ]
    )
    x_grid = np.array(
        [
            [
                x - 4 * space,
                x - 3 * space,
                x - 2 * space,
                x - space,
                x,
                x + space,
                x + 2 * space,
                x + 3 * space,
                x + 4 * space,
            ]
        ]
    ).T
    f_diff = np.sum(np.dot(coeff[order - 1, :], f(x_grid))) / space**order

    return f_diff

In [14]:
# Example of finite difference
my_func = lambda x: np.sin(x)

diff = finite_difference(my_func, 0)
diff

0.9999833334166665

# Numerical Integration

## Trapezoidal Rule

In [15]:
def trapezoidal(f, interval: nptyping.ArrayLike, num: int):
    grid = np.linspace(interval[0], interval[1], num)
    result = (f(grid[0]) / 2 + f(grid[-1]) / 2 + np.sum(f(grid[1:-1]))) * (
        grid[1] - grid[0]
    )

    return result

In [16]:
# Example of trapezoidal rule
import scipy.integrate as sciInt

my_func = lambda x: 4 * x / (2 - 8 * x**2)

area = trapezoidal(my_func, [3, 5], 51)
area2 = sciInt.quad(my_func, 3, 5)
print(area)
print(area2)

-0.25994825341093203
(-0.25994294716129407, 2.8859464502738915e-15)


## Simpson's Rule

In [17]:
def simpson(f, interval: nptyping.ArrayLike, num: int):
    if ~num % 2:
        raise Exception("Number of grid points should be odd.")
    grid = np.linspace(interval[0], interval[1], num)
    odd_terms = grid[1:-1:2]
    even_terms = grid[2:-1:2]
    result = (
        (
            f(grid[0])
            + f(grid[-1])
            + 4 * np.sum(f(odd_terms))
            + 2 * np.sum(f(even_terms))
        )
        * (grid[1] - grid[0])
        / 3
    )

    return result

In [18]:
# Example of trapezoidal rule
import scipy.integrate as sciInt

my_func = lambda x: 4 * x / (2 - 8 * x**2)

area = simpson(my_func, [3, 5], 51)
area2 = sciInt.quad(my_func, 3, 5)
print(area)
print(area2)

-0.25994294777388854
(-0.25994294716129407, 2.8859464502738915e-15)
